In [1]:
import sys
sys.path.append('../../build')
import libry as ry
import numpy as np
import time
import random

import cv2 as cv

In [2]:
np.random.seed(25)

RealWorld = ry.Config()

RealWorld.addFile("../../scenarios/challenge.g")
#Change color of objects
for o in range(0, 30):
    # color = list(np.random.choice(np.arange(0, 1, 0.05), size=2)) + [1]
    #color = list(np.random.choice(np.arange(0, 1, 0.01), size=3))
    color = random.choice([[0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0],
                           [1, 0.5, 0], [0.5, 0, 1], [0, 1, 0.5], [0, 0.5, 1], [0.5, 1, 0]])
    name = "obj%i" % o
    RealWorld.frame(name).setColor(color)

S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

q0 = C.getJointState()
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
L_gripper = C.frame("L_gripper")
L_gripper.setContact(1)

TODO:

0. add a second camera?
1. create a dict with ObjectID, Center of Object, Position and Orientation
2. - Also save which other objects "nearby" (maybe save distance between the centroids).
   - If the distance change the these are two different Objects and the connection can be deleted.
   - Otherwise save that this is "one object" just different sides of the Object
3. Track the objects all the time
4. Test grasping an object depending on the position (write a function for grasping where the input is
a position where the gripper has to go and his orientation. Or maybe just give an objects as input)

TODO later:
1. Sort the different Objects sizes
   - If the height and width of the object is too big (could not be grasped) - then mark of save in a list
   - Save which objects are one on another - maybe get the z Position and if z is heigher then th z of table it means one object is below
        - maybe the push the object down
   - (optional) if object is too far away, grasp another object and push the object nearer
2.depending on size of the object maybe do something like that:
   - push objects down, so that they are all on the table
   - objects which can be grasped, put them on a free space, to grasp them later
   - object which cannot be grasped push them towards the other hand and try to rotate them
        - also put these objects somewhere where is a free space
3 Stack objects one over another
     - sort by size --> biggest objects first later the smaller ones
     - calculate the rotation to put one edge on the table while the other side is still grasped. Then slide the gripper (halbkreis) out
     - check if center is on the right spot
     - maybe readjust with two hands the object if it is not on the right position.
     - stack as many objects as possible --> Retake the experiment 5 times with same objects and 3*5 with different ones --> evaluate

In [3]:
#the focal length
f = 0.895
f = f * 360.
#the relative pose of the camera
# pcl.setRelativePose('d(-90 0 0 1) t(-.08 .205 .115) d(26 1 0 0) d(-1 0 1 0) d(6 0 0 1) ')
fxfypxpy = [f, f, 320., 180.]


In [4]:
def detectObjects(rgb):

    #canny edge detection
    edges = cv.Canny(rgb, 20, 60)
    edges = cv.dilate(edges, None, iterations=1)
    edges = cv.erode(edges, None, iterations=1)

    #find contours in edges
    contours, hierarchy = cv.findContours(edges, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    good_contour = []   # good contours which are identified
    hull_list = []      #conved hull - the end points of rectangle

    for cnt in contours:
        #if small contour area - ignore
        if cv.contourArea(cnt) < 200:
            continue

        #Ignore the objects which are too far away or in the background
        mask = np.zeros(rgb.shape[:2], np.uint8)
        cv.drawContours(mask, cnt, -1, 255, 1)
        mean_color = cv.mean(rgb, mask=mask)
        mean_depth = cv.mean(depth, mask=mask)

        #compute the approx shape - try to fit Poly in shape
        approx = cv.approxPolyDP(cnt, 0.05 * cv.arcLength(cnt,True),True)


        if mean_color[2] > 165 and mean_depth[0] < 2 and len(approx) < 5:

            good_contour.append(cnt)
            #get convex hull
            hull = cv.convexHull(approx)
            hull_list.append(hull)

            # compute the center of the contour
            M = cv.moments(approx)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                # draw the contour and center of the shape on the image
                print(str(cX) + " " + str(cY))
                cv.circle(rgb, (cX, cY), 7, (255, 255, 255), -1)
                cv.putText(rgb, "center", (cX - 20, cY - 20),
                           cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

                cv.drawContours(rgb, hull, -1, (255, 0, 255), 3)

    return good_contour, edges


In [5]:
points = []
tau = .01
t = 0

while True:
    time.sleep(0.01)
    t += 1
    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
        cameraFrame.setPointCloud(points, rgb)
        V.recopyMeshes(C)
        V.setConfiguration(C)

        good_contours, edges = detectObjects(rgb=rgb)

        good = np.zeros(rgb.shape, np.uint8)
        cv.drawContours(good, good_contours, -1, (0, 255, 0), 1)

       # if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        if len(edges)>0: cv.imshow('OPENCV - edges', edges)
       # if len(good)>0: cv.imshow('OPENCV - depth', good)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    S.step([], tau, ry.ControlMode.none)






143 243
305 94
313 82
140 232
304 108
305 93
59 68
123 113
240 223
217 210
167 211
118 233
171 171
88 136
304 108
305 93
507 151
629 316
242 231
185 215
197 178
468 189
304 108
305 93
243 239
141 244
191 216
210 180
80 156
62 162
304 108
305 93
25 281
243 239
304 108
305 93
243 239
193 215
212 184
304 108
305 93
241 234
185 216
203 185
304 108
305 93
219 226
173 217
182 185
304 108
305 93
169 204
304 108
305 93
19 311
132 251
195 236
140 209
304 108
305 93
20 310
90 246
167 208
205 212
304 108
305 93
20 310
186 246
89 248
125 229
164 210
204 212
304 108
305 93
20 311
186 246
89 248
125 229
164 210
204 212
304 108
305 93
20 311
186 246
89 248
125 229
164 210
204 212
304 108
305 93
20 311
186 246
89 248
125 229
164 210
204 212
304 108
305 93
20 311
186 246
89 248
125 229
164 210
204 212
304 108
305 93
20 311
186 246
89 248
125 229
164 210
204 212
304 108
305 93
20 311
186 246
89 248
125 229
164 210
204 212
304 108
305 93
20 311
186 246
89 248
125 229
164 210
204 212
304 108
305 93
20 311